# Domínios 3 e 4 — Engenharia do Sistema e Análise de Performance

Este notebook demonstra as etapas de engenharia do sistema fotovoltaico (Domínio 3) e de avaliação de performance (Domínio 4) utilizando o ecossistema **pvlib**.
Partimos do pressuposto de que os resultados dos Domínios 1 e 2 (localização, dados meteorológicos e posição solar) já estão disponíveis.

## Pré-requisitos e contexto

* Localização, dados meteorológicos (TMY) e posição solar foram obtidos anteriormente.
* Utilizamos um módulo bifacial e um inversor equivalentes aos informados no JSON original.
* O notebook expande o fluxo padrão incorporando ganhos bifaciais, perdas por sujidade e perdas em transformador.

In [ ]:
# CÉLULA 1 — Setup do ambiente e carregamento dos dados de entrada
%pip install pvlib pandas matplotlib --quiet

import pandas as pd
import pvlib
import matplotlib.pyplot as plt

# --- DADOS PRÉ-PROCESSADOS (Resultados dos Domínios 1 e 2) ---
latitude, longitude = -23.55, -46.63  # São Paulo, SP
location = pvlib.location.Location(latitude, longitude, tz='America/Sao_Paulo')

# O TMY do PVGIS é utilizado como exemplo de dados climáticos horários.
weather, _, _, _ = pvlib.iotools.get_pvgis_tmy(latitude, longitude, map_variables=True)
times = weather.index
solar_position = location.get_solarposition(times)

print("Ambiente e dados de entrada (Domínios 1-2) prontos para a análise.")

## Domínio 3 — Modelagem da geração e conversão de energia

Selecionamos um módulo bifacial de características semelhantes ao equipamento informado e o inversor correspondente.
A **ModelChain** integra os modelos óticos, térmicos e elétricos para obter a potência AC entregue pelo inversor.

In [ ]:
# CÉLULA 2 — Parametrização dos componentes e simulação do balanço energético
from pvlib.pvsystem import PVSystem, FixedMount
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
from pvlib.modelchain import ModelChain

# Catálogos CEC com especificações de módulos e inversores
cec_modules = pvlib.pvsystem.retrieve_sam('CECMod')
module = cec_modules['LG_Electronics_Inc__LG455N2W-A5']

cec_inverters = pvlib.pvsystem.retrieve_sam('CECInverter')
inverter = cec_inverters['Huawei_Technologies_Co___Ltd___SUN2000_6KTL_L1']

# Parâmetros térmicos representativos de um módulo bifacial vidro/vidro
temp_params = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

mount = FixedMount(surface_tilt=23.5, surface_azimuth=180)

system = PVSystem(
    mounts=[mount],
    module_parameters=module,
    inverter_parameters=inverter,
    temperature_model_parameters=temp_params,
    modules_per_string=10,
    strings_per_inverter=1,
)

mc = ModelChain(system, location)
mc.run_model(weather)

ac_power = mc.results.ac.fillna(0)
print("Módulo selecionado:", module.name)
print("Inversor selecionado:", inverter.name)
print("
--- Domínio 3: Resultados da Simulação (AC positivo) ---")
print(ac_power[ac_power > 0].head())

plt.figure(figsize=(12, 5))
ac_power.loc['2023-01-01':'2023-01-07'].plot(color='tab:blue')
plt.title('Produção de energia AC simulada — primeira semana do ano')
plt.ylabel('Potência (W)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Domínio 4 — Ganhos bifaciais, perdas sistêmicas e indicadores

Esta etapa incorpora efeitos adicionais (bifacialidade, sujidade e perdas em transformador) e calcula indicadores de desempenho como PR e fator de capacidade.

In [ ]:
# CÉLULA 3 — Quantificação de ganhos e perdas sistêmicas
import numpy as np

# Ganho por bifacialidade considerando o método de sheds infinitos
bifaciality = float(module.get('Bifaciality', module.get('bifaciality', 0.75)))
bifacial_irrad = pvlib.bifacial.infinite_sheds(
    surface_tilt=mount.surface_tilt,
    surface_azimuth=mount.surface_azimuth,
    solar_zenith=solar_position['apparent_zenith'],
    solar_azimuth=solar_position['azimuth'],
    dni=weather['dni'],
    dhi=weather['dhi'],
    ghi=weather['ghi'],
    albedo=0.25,
)

poa_front = bifacial_irrad['poa_front']
poa_back = bifacial_irrad['poa_back']
poa_bifacial = poa_front + poa_back * bifaciality
poa_effective = poa_bifacial.copy()

# Perfil de perdas por sujidade (mensal) aplicado sobre a irradiância efetiva
monthly_soiling_loss = [0.5, 0.7, 1.0, 1.5, 2.0, 2.5, 3.0, 2.5, 2.0, 1.0, 0.5, 0.5]  # %
loss_map = {month: loss for month, loss in enumerate(monthly_soiling_loss, start=1)}
soiling_profile = times.month.map(loss_map) / 100.0
soiling_factor = 1 - soiling_profile
poa_effective *= soiling_factor

# Ajustes na produção AC pela combinação dos ganhos e perdas
poa_front_safe = poa_front.replace(0, np.nan)
bifacial_gain = (poa_bifacial / poa_front_safe).replace([np.inf, -np.inf], np.nan).fillna(1.0)
ac_with_bifacial = ac_power * bifacial_gain
ac_after_soiling = ac_with_bifacial * soiling_factor

transformer_losses = pvlib.transformer.losses(
    ac_after_soiling.fillna(0),
    p_ac_nom=float(inverter['Paco']) * 0.95,
    p_loss_nom=float(inverter['Paco']) * 0.02,
    p_loss_vac=float(inverter['Paco']) * 0.005,
)

final_ac_power = (ac_after_soiling - transformer_losses).clip(lower=0)

print("--- Domínio 4: Ganhos e perdas sistêmicas ---")
print(f"Ganho médio por bifacialidade: {(bifacial_gain.mean() - 1) * 100:.2f}%")
print(f"Perda média por sujidade: {soiling_profile.mean() * 100:.2f}%")
ac_after_soiling_sum = ac_after_soiling.sum()
if ac_after_soiling_sum > 0:
    transformer_loss_share = transformer_losses.sum() / ac_after_soiling_sum
else:
    transformer_loss_share = 0.0
print(f"Perda média no transformador (potência): {transformer_loss_share * 100:.2f}%")

In [ ]:
# CÉLULA 4 — Indicadores de performance (KPIs)
def energy_from_power(series):
    '''Converte uma série de potência (W) em energia (kWh).'''
    if series.empty:
        return 0.0
    time_step_hours = series.index.to_series().diff().dt.total_seconds() / 3600.0
    if len(time_step_hours) > 1:
        time_step_hours.iloc[0] = time_step_hours.iloc[1]
    else:
        time_step_hours.iloc[0] = 1.0
    energy_wh = (series.fillna(0) * time_step_hours.fillna(1.0)).sum()
    return energy_wh / 1000.0

poa_energy = energy_from_power(poa_effective)
ac_energy_before_transformer = energy_from_power(ac_after_soiling)
ac_energy_final = energy_from_power(final_ac_power)
transformer_energy_loss = energy_from_power(transformer_losses)

impo = float(module.get('Impo', module.get('Imref')))
vmpo = float(module.get('Vmpo', module.get('Vmpref')))
p_dc_nominal = system.modules_per_string * system.strings_per_inverter * impo * vmpo / 1000.0

time_step_hours = times.to_series().diff().dt.total_seconds() / 3600.0
if len(time_step_hours) > 1:
    time_step_hours.iloc[0] = time_step_hours.iloc[1]
else:
    time_step_hours.iloc[0] = 1.0
hours_in_year = time_step_hours.sum()

if p_dc_nominal > 0 and poa_energy > 0:
    performance_ratio = (ac_energy_before_transformer / (p_dc_nominal * poa_energy)) * 100.0
else:
    performance_ratio = 0.0

if p_dc_nominal > 0 and hours_in_year > 0:
    capacity_factor = (ac_energy_final / (p_dc_nominal * hours_in_year)) * 100.0
else:
    capacity_factor = 0.0

if ac_energy_before_transformer > 0:
    transformer_loss_pct = (transformer_energy_loss / ac_energy_before_transformer) * 100.0
else:
    transformer_loss_pct = 0.0

print("--- Domínio 4: Indicadores de Performance ---")
print(f"Energia AC anual final: {ac_energy_final:.2f} kWh")
print(f"Irradiação efetiva anual (POA): {poa_energy:.2f} kWh/m²")
print(f"Performance Ratio (PR): {performance_ratio:.2f} %")
print(f"Capacity Factor (CF): {capacity_factor:.2f} %")
print(f"Energia perdida no transformador: {transformer_energy_loss:.2f} kWh ({transformer_loss_pct:.2f} %)")